In [5]:
# Step 1: Import Library
import urllib.request
import zipfile
import tensorflow as tf
import os
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop


In [6]:
# Step 2: Download dan Ekstrak Dataset
train_url = 'https://github.com/dicodingacademy/assets/releases/download/release-horse-or-human/horse-or-human.zip'
val_url = 'https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/validation-horse-or-human.zip'

urllib.request.urlretrieve(train_url, 'horse-or-human.zip')
urllib.request.urlretrieve(val_url, 'validation-horse-or-human.zip')

with zipfile.ZipFile('horse-or-human.zip', 'r') as zip_ref:
    zip_ref.extractall('data/horse-or-human')

with zipfile.ZipFile('validation-horse-or-human.zip', 'r') as zip_ref:
    zip_ref.extractall('data/validation-horse-or-human')


In [7]:
# Step 3: Setup ImageDataGenerator
TRAINING_DIR = 'data/horse-or-human'
VALIDATION_DIR = 'data/validation-horse-or-human'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)


Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [8]:
# Step 4: Membangun Model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Jangan ubah baris ini!
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
# Step 5: Compile Model
model.compile(
    loss='binary_crossentropy',
    optimizer=RMSprop(learning_rate=0.001),
    metrics=['accuracy']
)


In [10]:
# Step 6: Train Model
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=validation_generator,
    verbose=1
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.5575 - loss: 0.9314 - val_accuracy: 0.5469 - val_loss: 0.6273
Epoch 2/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 44s 1s/step - accuracy: 0.7514 - loss: 0.5370 - val_accuracy: 0.7695 - val_loss: 0.7389
Epoch 3/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.7819 - loss: 0.5174 - val_accuracy: 0.6562 - val_loss: 2.3414
Epoch 4/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.8631 - loss: 0.3877 - val_accuracy: 0.6016 - val_loss: 3.1999
Epoch 5/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.8854 - loss: 0.2799 - val_accuracy: 0.6797 - val_loss: 2.1158
Epoch 6/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.9212 - loss: 0.2242 - val_accuracy: 0.7383 - val_loss: 1.3612
Epoch 7/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.9343 - loss: 0.1877 - val_accuracy: 0.7188 - val_loss: 1.8281
Epoch 8/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.9428 - loss: 0.1435 - val_accuracy: 0.7031 - val_loss:

In [11]:
# Step 7: Simpan Model
model.save("model_05.h5")